# Tutorial 6:  Size and colour reference

Unless images are taken in a highly standardized environment, e.g. via a scanner or a microscope, variation will be introduced in terms of exposure or distance between camera and photographed object, zooming, etc. To compensate this variation among images within and across datasets, Phenopype contains some preprocessing tools that can automatically correct images.  

<div class="alert alert-block alert-info">
<div class="admonition seealso">
<p class="admonition-title" > See also:</p>
<p>

*   [phenopype docs: Project-API](https://www.phenopype.org/docs/api/project/)
*   [phenopype docs: preprocessing-API](https://www.phenopype.org/docs/api/core/#phenopype.core.preprocessing.detect_reference)

</p>
</div>
</div>
    



## Load project 

We will use the project we created before in [Tutorial 5](tutorial_4.ipynb):

In [5]:
import phenopype as pp
import os

os.chdir(r"D:\workspace\git-repos\phenopype-tutorials")

myproj = pp.Project(r"_temp/tutorial_project") 

--------------------------------------------
Found existing project root directory - loading from:
D:\workspace\git-repos\phenopype-tutorials\_temp\tutorial_project

Project "tutorial_project" successfully loaded with 3 images
--------------------------------------------


First we will need to select and measure the pixel-ratio in a reference image, to which we want to match all images in the project.  

## Adding a reference to the project

With the class method `add_reference` we can set a project specific scale by measuring the pixel-ratio in a reference image. The steps for this are:

1. Click on two points with a known distance, e.g. on a piece of mm-paper that you put in the image, and hit `Enter`.
2. Enter the length that will be used to calculate the  pixel-ratio.
3. Mark the boundaries of the reference card to create a template for automatic scale detection that is saved on the project's root directory (in the template folder).



<br>

<center>
<div style="text-align: center; width: 600px;" >
<img src="_assets/images/scale.gif" style="height: auto; max-width: 100%;">
</div>
<div style="text-align: left; width: 600px;" >
    
**Fig 1** The reference image can be any image, but choose it carefully: if you plan on doing brightness and colour corrections, it should be in the middle of the distribution of all exposures and colours so corrections will not over-expose or over-saturate the images. 
    
</div>
</center>


We will use the image `stickleback_side.jpg` from the `image` folder in `tutorials`:

In [7]:
myproj.add_reference(
    reference_image_path=r"tutorials/images/stickleback_side.jpg", 
    reference_tag="scale1",
)

Reference set
Reference image saved under D:\workspace\git-repos\phenopype-tutorials\_temp\tutorial_project\reference\scale1_full_image.tif
Reference image saved under D:\workspace\git-repos\phenopype-tutorials\_temp\tutorial_project\reference\scale1_search_template.tif
Saved reference info to project attributes.
setting active global project reference to "scale1" for 0__stickle1 (active=True)
setting active global project reference to "scale1" for 0__stickle2 (active=True)
setting active global project reference to "scale1" for 0__stickle3 (active=True)


We are now all set to automatically detect the reference card in our images.

## Detecting a reference

To detect the reference template in our images, we need the function `detect_reference` from the `preprocessing` module. To do so on the existing project, we will use the `edit_config` method that was demonstrated in [Tutorial 5](tutorial_5.ipynb#Modify-configurations-in-all-subdirectories). Again, we want to modify the config in two places: first in the preprocessing chunk, to perform the actual reference detection, and then in the visualization chunk, to show the outline of the detected reference:

In [8]:
## modify the "preprocessing" section

target1 = """    - preprocessing:"""
replacement1 = """    - preprocessing:
        - detect_reference"""
    
myproj.edit_config(
    tag="plates-v1",
    target=target1,
    replacement=replacement1,
)

config_info:
    config_name: pype_config_plates-v1.yaml
    date_created: '2021-12-25 12:52:19'
    date_last_modified:
    template_name: quickstart.yaml
    template_path: D:\workspace\git-repos\phenopype-templates\templates\various\quickstart.yaml
processing_steps:
    - preprocessing:
        - detect_reference
        - create_mask:
            ANNOTATION: {type: mask, id: a, edit: false}
            tool: polygon
        - blur:
            kernel_size: 7
    - segmentation:
        - threshold:
            method: adaptive
            blocksize: 299
            constant: 5
            channel: red
        - morphology:
            operation: open
            shape: ellipse
            kernel_size: 3
            iterations: 2
        - detect_contour:
            ANNOTATION: {type: contour, id: a, edit: overwrite}
    - segmentation:
        - edit_contour:
            ANNOTATION: {type: drawing, id: a, edit: false}
            overlay_blend: 0
        - detect_contour:
        

In [10]:
## "visualization" modification to add `draw_reference`:

target2 = """    - visualization:"""
replacement2 = """    - visualization:
        - draw_reference:
            line_colour: aqua
            line_width: 10
            scale: true"""
    
myproj.edit_config(
    tag="plates-v1",
    target=target2,
    replacement=replacement2,
)

config_info:
    config_name: pype_config_plates-v1.yaml
    date_created: '2021-12-25 12:52:19'
    date_last_modified:
    template_name: quickstart.yaml
    template_path: D:\workspace\git-repos\phenopype-templates\templates\various\quickstart.yaml
processing_steps:
    - preprocessing:
        - detect_reference:
            ANNOTATION: {type: reference, id: a, edit: false}
        - create_mask:
            ANNOTATION: {type: mask, id: a, edit: false}
            tool: polygon
        - blur:
            kernel_size: 7
    - segmentation:
        - threshold:
            method: adaptive
            blocksize: 299
            constant: 5
            channel: red
        - morphology:
            operation: open
            shape: ellipse
            kernel_size: 3
            iterations: 2
        - detect_contour:
            ANNOTATION: {type: contour, id: a, edit: overwrite}
    - segmentation:
        - edit_contour:
            ANNOTATION: {type: drawing, id: a, edit: false}


Now we run our loop with the new `pype` configuration:  

In [11]:
for path in myproj.dir_paths:
    pp.Pype(path, tag="plates-v1")

Format path to abspath
- no annotation_type selected - returning all annotations

AUTOLOAD
- annotations loaded:
{
"contour": ["a", "b"],
"drawing": ["a"],
"landmark": ["a"],
"mask": ["a"],
"morphology": ["a"],
"reference": ["a"]
}
- reference template image loaded from root directory


------------+++ new pype iteration 2021:12:26 21:13:06 +++--------------




PREPROCESSING
detect_reference
- loaded existing annotation of type "reference" with ID "a": skipping (edit=False)
create_mask
- loaded existing annotation of type "mask" with ID "a": skipping (edit=False)
blur


SEGMENTATION
threshold
- decompose image: using red channel
- including pixels from 1 drawn masks 
- excluding pixels from reference
morphology
detect_contour
- loaded existing annotation of type "contour" with ID "a": overwriting (edit=overwrite)
- did not find contours that match criteria


SEGMENTATION
edit_contour
- loaded existing annotation of type "drawing" with ID "a": skipping (edit=False)
- "contour_id" not p



EXPORT
save_canvas
- image saved under D:\workspace\git-repos\phenopype-tutorials\_temp\tutorial_project\data\0__stickle3\canvas_plates-v1.jpg (overwritten).
save_annotation
- loading existing annotation file
- updating annotation of type "contour" with id "a" in "annotations_plates-v1.json" (overwrite="entry")
- updating annotation of type "contour" with id "b" in "annotations_plates-v1.json" (overwrite="entry")
- updating annotation of type "drawing" with id "a" in "annotations_plates-v1.json" (overwrite="entry")
- updating annotation of type "landmark" with id "a" in "annotations_plates-v1.json" (overwrite="entry")
- updating annotation of type "mask" with id "a" in "annotations_plates-v1.json" (overwrite="entry")
- updating annotation of type "morphology" with id "a" in "annotations_plates-v1.json" (overwrite="entry")
- writing annotation of type "reference" with id "a" to "annotations_plates-v1.json"


------------+++ finished pype iteration +++--------------
-------(End with Ct

In [13]:
myproj.collect_results(tag="plates-v1",          
                       files="canvas", # 
                       folder="canvas-v1",
                       overwrite=True)

Created D:\workspace\git-repos\phenopype-tutorials\_temp\tutorial_project\results\canvas-v1
Search string: ['canvas_plates-v1']
Collected canvas_plates-v1.jpg from 0__stickle1
0__stickle1_canvas_plates-v1.jpg saved under D:\workspace\git-repos\phenopype-tutorials\_temp\tutorial_project\results\canvas-v1\0__stickle1_canvas_plates-v1.jpg.
Collected canvas_plates-v1.jpg from 0__stickle2
0__stickle2_canvas_plates-v1.jpg saved under D:\workspace\git-repos\phenopype-tutorials\_temp\tutorial_project\results\canvas-v1\0__stickle2_canvas_plates-v1.jpg.
Collected canvas_plates-v1.jpg from 0__stickle3
0__stickle3_canvas_plates-v1.jpg saved under D:\workspace\git-repos\phenopype-tutorials\_temp\tutorial_project\results\canvas-v1\0__stickle3_canvas_plates-v1.jpg.
